In [ ]:
import numpy as np
import pandas as pd
import os
from os import path
from tqdm import tqdm
import matplotlib.pyplot as plt
from scipy import stats


In [ ]:
project_root = '/home/project/gas_anormaly_detection/restaurant/1training/'

In [ ]:
data_root = path.join(project_root, 'Dataset')
save_data_root = path.join(project_root, '2pattern_extraction/save_data')
if not path.exists(save_data_root):
    os.mkdir(save_data_root)
    print("make a new dir.")

In [ ]:
hourly_week_data_path = path.join(data_root, 'hourly_week_data_afterrules')
hourly_week_date_path = path.join(data_root, 'hourly_week_date_afterrules')

hourly_week_data_list = os.listdir(hourly_week_data_path)

In [ ]:
saved_sample = []

other_sample_index = 0
other_sample = []
other_sample_corr = []


for f_name in tqdm(hourly_week_data_list):
    id_ = str(os.path.splitext(f_name)[0])
    hourly_week_data = pd.read_csv(path.join(hourly_week_data_path, f_name))
        
    if len(hourly_week_data)<3:
        for row_num in range(len(hourly_week_data)):
            this_week = np.array(hourly_week_data.ix[row_num])
            saved_sample.append(this_week)
            
    else:
        
        for row_num in range(len(hourly_week_data)):
            this_week = np.array(hourly_week_data.ix[row_num])

            if row_num == 0:
                next_week = np.array(hourly_week_data.ix[row_num+1])
                cor = stats.pearsonr(this_week, next_week)[0]
            elif row_num == (len(hourly_week_data)-1):
                pre_week = np.array(hourly_week_data.ix[row_num-1])
                cor = stats.pearsonr(this_week, pre_week)[0]
            else:
                next_week = np.array(hourly_week_data.ix[row_num+1])
                pre_week = np.array(hourly_week_data.ix[row_num-1])
                cor = (stats.pearsonr(this_week, next_week)[0]+stats.pearsonr(this_week,pre_week)[0])/2
            other_sample.append(this_week)
            other_sample_corr.append(cor)

            other_sample_index+=1

In [ ]:
other_sample = np.array(other_sample)
other_sample_corr = np.array(other_sample_corr)

In [ ]:
plt.rcParams['figure.figsize']=(24.0,6.0)

plt.hist(other_sample_corr, bins=10)
plt.ylabel('num')
plt.xlabel('correlation')
plt.show()


In [ ]:
abnorm_index = np.where(other_sample_corr<0.3)[0]

abnorm_sample_np = other_sample[abnorm_index]
nomal_index = [val for val in range(len(other_sample)) if val not in abnorm_index]

normal_sample_np = other_sample[nomal_index]
normal_sample_np = np.concatenate((normal_sample_np,np.array(saved_sample)),axis=0)

In [ ]:
# save normal and abnormal set
np.save(path.join(save_data_root, 'abnorm_sample_intra.npy'), abnorm_sample_np)
np.save(path.join(save_data_root, 'norm_sample_intra.npy'), normal_sample_np)